### Multimodal Model: Early Fusion (CLIP + DeBERTa)

The goal of this notebook is to run an ablation study to determine how much impact does the image and text has in the multimodal model. 

In [1]:
#Libraries
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score,confusion_matrix, classification_report
from tqdm.auto import tqdm
from PIL import Image
from transformers import AutoTokenizer,AutoModel,CLIPModel,CLIPProcessor,get_linear_schedule_with_warmup
import warnings
warnings.filterwarnings('ignore')


# Random seed for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

# Device configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Seed: {SEED}")
print(f"Using device: {DEVICE}")

Seed: 42
Using device: cuda


In [2]:
#Paths
DATA_PATH = "../../../data/"
IMG_PATH = "../../../data/images"
OUTPUT_DIR = "../../results/multimodal/baseline_multimodal/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

train_path = os.path.join(DATA_PATH,"train_augmented.csv")
dev_path   = os.path.join(DATA_PATH,"dev.csv")
test_path  = os.path.join(DATA_PATH,"test.csv")

#Load Data
df_train = pd.read_csv(train_path)
df_dev   = pd.read_csv(dev_path)
df_test  = pd.read_csv(test_path)

# Map labels to ints
stance_2id = {"oppose": 0, "support": 1}
pers_2id = {"no": 0, "yes": 1}

for df in [df_train, df_dev, df_test]:
    df["label"] = df["stance"].map(stance_2id)
    df["persuasiveness_label"] = df["persuasiveness"].map(pers_2id)


print(f"\n Train label distribution:")
print(f"\n Stance: \n Oppose: {(df_train['label']==0).sum()}\n Support: {(df_train['label']==1).sum()}")
print(f"\n\n  Persuasiveness \n No: {(df_train['persuasiveness_label']==0).sum()}\n Yes: {(df_train['persuasiveness_label']==1).sum()}")


df_train.head()


 Train label distribution:

 Stance: 
 Oppose: 1095
 Support: 1095


  Persuasiveness 
 No: 1548
 Yes: 642


,tweet_id,tweet_url,tweet_text,stance,persuasiveness,split,label,persuasiveness_label
0,1148501065308004357,https://t.co/VQP1FHaWAg,Let's McGyver some Sanity in America!\n\nYou a...,support,no,train,1,0
1,1103872992537276417,https://t.co/zsyXYSeBkp,A child deserves a chance at life. A child des...,oppose,no,train,0,0
2,1151528583623585794_aug,https://t.co/qSWvDX5MnM,"Dear prolifers: girls as young as 10, 11, 12 a...",support,no,train,1,0
3,1100166844026109953,https://t.co/hxH8tFIHUu,The many States will attempt to amend their co...,support,no,train,1,0
4,1021830413550067713,https://t.co/5whvEEtoQR,"Every #abortion is wrong, no matter what metho...",oppose,yes,train,0,1


In [3]:
#Models
TEXT_MODEL_NAME = "microsoft/deberta-v3-base"
VISION_MODEL_NAME = "openai/clip-vit-base-patch32" 

In [4]:
 # Training hyperparameters
BATCH_SIZE = 16
NUM_EPOCHS = 15
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 1e-4
WARMUP_RATIO = 0.1

# Early stopping
PATIENCE = 5

# Text preprocessing
MAX_TEXT_LENGTH = 105

# Other
NUM_WORKERS = 1
PIN_MEMORY = True if torch.cuda.is_available() else False
os.makedirs(OUTPUT_DIR, exist_ok=True)

###  Multimodal Dataset
We create a MultimodalDataset that will return:
- tokenized text (input_ids, attention_mask)
- image (Pixel Values PIL)
- label (stance)


We will handle corrupted images safely (Grey image).

In [5]:
class MultimodalDatasetCLIP(Dataset):
    """
    Dataset for CLIP + DeBERTa multimodal learning.
    Returns tokenized text + processed image + label.
    """
    def __init__(self, dataframe: pd.DataFrame, img_dir: str, tokenizer, processor, max_length: int = 128):

        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.processor = processor 
        self.max_length = max_length

        # Class distribution
        self.class_counts = self.df['label'].value_counts().to_dict()
        self.num_samples = len(self.df)
        print(f"  Dataset created: {self.num_samples} samples")
        print(f"    - Class 0 (oppose):  {self.class_counts.get(0, 0)}")
        print(f"    - Class 1 (support): {self.class_counts.get(1, 0)}")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load Image
        img_path = os.path.join(self.img_dir, str(row['tweet_id']) + ".jpg")
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            # fallback to grey image
            image = Image.new("RGB", (224, 224), color=(128, 128, 128))

        # Text
        text = str(row['tweet_text'])
        encoding = self.tokenizer(text,add_special_tokens=True,
                                  max_length=self.max_length,padding='max_length',
                                  truncation=True,return_tensors='pt')

        # Label
        label = row['label']

        return {
            'pixel_values': image,
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long),
            'tweet_id': str(row['tweet_id']),
            'text': text
        }

    def get_class_weights(self, device):
        num_class_0 = self.class_counts.get(0, 0)
        num_class_1 = self.class_counts.get(1, 0)
        total = self.num_samples
        weights = torch.tensor([
            total / num_class_0 if num_class_0 > 0 else 1.0,
            total / num_class_1 if num_class_1 > 0 else 1.0], dtype=torch.float32).to(device)
        return weights

In [6]:
# Collate function for DataLoader
def collate_fn_clip(batch, processor):
    images = [item['pixel_values'] for item in batch]
    labels = torch.stack([item['label'] for item in batch])
    input_ids = torch.stack([item['input_ids'] for item in batch])
    attention_mask = torch.stack([item['attention_mask'] for item in batch])

    # Process images
    processed = processor(images=images, return_tensors="pt")
    
    return {
        'pixel_values': processed['pixel_values'],  # tensor [B, 3, 224, 224]
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels}


In [7]:
# Tokenizer and Processor
tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_NAME)
clip_processor = CLIPProcessor.from_pretrained(VISION_MODEL_NAME)
print("Tokenizer and Processor loaded.\n")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Tokenizer and Processor loaded.



In [8]:
train_dataset = MultimodalDatasetCLIP(df_train, IMG_PATH, tokenizer, clip_processor, MAX_TEXT_LENGTH)
dev_dataset   = MultimodalDatasetCLIP(df_dev, IMG_PATH, tokenizer, clip_processor, MAX_TEXT_LENGTH)
test_dataset  = MultimodalDatasetCLIP(df_test, IMG_PATH, tokenizer, clip_processor, MAX_TEXT_LENGTH)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS,
                          pin_memory=PIN_MEMORY, collate_fn=lambda batch: collate_fn_clip(batch, clip_processor))
dev_loader = DataLoader(dev_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS,
                        pin_memory=PIN_MEMORY, collate_fn=lambda batch: collate_fn_clip(batch, clip_processor))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS,
                         pin_memory=PIN_MEMORY, collate_fn=lambda batch: collate_fn_clip(batch, clip_processor))


  Dataset created: 2190 samples
    - Class 0 (oppose):  1095
    - Class 1 (support): 1095
  Dataset created: 200 samples
    - Class 0 (oppose):  127
    - Class 1 (support): 73
  Dataset created: 300 samples
    - Class 0 (oppose):  182
    - Class 1 (support): 118


In [9]:
#We test our datasets and loaders
sample = train_dataset[0]
print(f"Sample text shape: {sample['input_ids'].shape}")
print(f"Sample label: {sample['label']}")
print(f"Sample image type: {type(sample['pixel_values'])}")

Sample text shape: torch.Size([105])
Sample label: 1
Sample image type: <class 'PIL.Image.Image'>


In [10]:
class MultimodalBaseline(nn.Module):
    def __init__(
        self,
        text_model_name="microsoft/deberta-v3-base",
        vision_model_name="openai/clip-vit-base-patch32",
        num_classes=2,
        freeze_text=False,
        freeze_vision=True,
        fusion_type="concat"
    ):
        super().__init__()
        self.fusion_type = fusion_type

        # TEXT ENCODER (DEBERTA)
        print(f"Loading TEXT encoder: {text_model_name}")
        self.text_encoder = AutoModel.from_pretrained(text_model_name)
        self.text_hidden = self.text_encoder.config.hidden_size  #768

        if freeze_text:
            for p in self.text_encoder.parameters():
                p.requires_grad = False
            print("Text encoder FROZEN")

        # VISION ENCODER (CLIP)
        print(f"Loading VISION encoder: {vision_model_name}")
        self.clip_model = CLIPModel.from_pretrained(vision_model_name)
        self.vision_model = self.clip_model.vision_model
        self.clip_hidden = self.clip_model.config.projection_dim  # 512

        if freeze_vision:
            for p in self.vision_model.parameters():
                p.requires_grad = False
            # Optionally unfreeze last layer
            for name, param in self.vision_model.named_parameters():
                if "encoder.layers.11" in name:
                    param.requires_grad = True
            print("CLIP encoder partially frozen (last layer trainable)")


        # Project vision embeddings to text_hidden (needed for mean & gated & proj_concat)
        self.vision_proj = nn.Linear(self.clip_hidden, self.text_hidden)

        # Gated fusion
        if fusion_type == "gated":
            self.gate = nn.Sequential(
                nn.Linear(self.text_hidden * 2, self.text_hidden),
                nn.Sigmoid())

        # CLASSIFIER
        if fusion_type == "concat":
            fused_dim = self.text_hidden + self.clip_hidden
        elif fusion_type == "mean":
            fused_dim = self.text_hidden
        elif fusion_type == "gated":
            fused_dim = self.text_hidden
        elif fusion_type == "proj_concat":
            fused_dim = self.text_hidden * 2
        else:
            raise ValueError(f"Unknown fusion_type: {fusion_type}")

        self.classifier = nn.Linear(fused_dim, num_classes)
        print(f"MultimodalBaseline initialized | Fusion = {fusion_type}")
        print(f"  Fused embedding size: {fused_dim}")

    def forward(self, input_ids, attention_mask, images=None, mode="multimodal"):
        
        # TEXT EMBEDDINGS
        text_out = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = text_out.last_hidden_state[:, 0, :]  # CLS token [B, text_hidden]

        # VISION EMBEDDINGS
        if mode == "multimodal" and images is not None:
            vision_feat = self.vision_model(pixel_values=images)
            cls_embedding = vision_feat.last_hidden_state[:, 0, :]  # CLS token
            vision_emb = self.clip_model.visual_projection(cls_embedding)  # [B, clip_hidden]
        else:
            vision_emb = torch.zeros(text_emb.size(0), self.clip_hidden, device=text_emb.device)

        # FUSION (EARLY)
        if self.fusion_type == "concat":
            fused = torch.cat([text_emb, vision_emb], dim=1)
        elif self.fusion_type == "mean":
            vision_emb_proj = self.vision_proj(vision_emb)  # [B, text_hidden]
            # Dentro del forward, justo antes de fusionar:
            print("text_emb mean abs:", text_emb.abs().mean().item())
            print("vision_emb mean abs:", vision_emb.abs().mean().item())
            print("vision_emb std:", vision_emb.std().item())
            fused = (text_emb + vision_emb_proj) / 2
        elif self.fusion_type == "gated":
            vision_emb_proj = self.vision_proj(vision_emb)  # [B, text_hidden]
            gate_input = torch.cat([text_emb, vision_emb_proj], dim=1)  # [B, 2*text_hidden]
            gate = self.gate(gate_input)  # [B, text_hidden], sigmoid outputs 0-1
            fused = gate * text_emb + (1 - gate) * vision_emb_proj  # [B, text_hidden]
        elif self.fusion_type == "proj_concat":
            fused = torch.cat([text_emb, self.vision_proj(vision_emb)], dim=1)
        else:
            raise ValueError(f"Unknown fusion_type: {self.fusion_type}")

        # CLASSIFIER
        logits = self.classifier(fused)
        return logits


### Training 

In [11]:
def train_multimodal_model(
    model, 
    train_loader, 
    dev_loader,
    num_epochs=15, 
    learning_rate=2e-5,
    weight_decay=1e-4, 
    warmup_ratio=0.1,
    mode="multimodal",
    patience=5, 
    device=DEVICE):
    model = model.to(device)

    # CLASS WEIGHTS (IMBALANCE)
    class_weights = train_loader.dataset.get_class_weights(device)
    print(
        f"Class weights: "
        f"oppose={class_weights[0]:.3f}, "
        f"support={class_weights[1]:.3f}"
    )

    criterion = nn.CrossEntropyLoss(weight=class_weights)

    # OPTIMIZER (Per encoder and classifier)
    optimizer = torch.optim.AdamW(
        [
            {"params": model.text_encoder.parameters(),"lr": learning_rate},
            {"params": model.vision_model.parameters(),"lr": learning_rate * 0.5},
            {"params": model.classifier.parameters(),"lr": learning_rate * 2}
            ],
        weight_decay=weight_decay)

    # SCHEDULER
    num_training_steps = len(train_loader) * num_epochs
    num_warmup_steps = int(num_training_steps * warmup_ratio)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps
    )

    print(f"Total training steps: {num_training_steps}")
    print(f"Warmup steps: {num_warmup_steps}")

    # HISTORY
    history = {
        "train_loss": [],
        "train_f1": [],
        "dev_loss": [],
        "dev_f1": [],
        "learning_rates": []
    }

    # EARLY STOPPING
    best_f1 = 0.0
    best_model_state = None
    epochs_without_improvement = 0

    # TRAINING LOOP
    for epoch in range(num_epochs):
        print(f"\n{'=' * 60}")
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"{'=' * 60}")

        # -------- TRAIN --------
        model.train()
        train_loss = 0.0
        train_preds, train_labels = [], []

        for batch in tqdm(train_loader, desc="Training"):
            optimizer.zero_grad()

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            if mode == "text_only":
                images = None

            logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                images=images,
                mode=mode
            )

            loss = criterion(logits, labels)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            train_loss += loss.item() * labels.size(0)
            preds = torch.argmax(logits, dim=1)

            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_loss /= len(train_loader.dataset)
        train_f1 = f1_score(
            train_labels,
            train_preds,
            average="binary",
            pos_label=1,
            zero_division=0
        )

        # -------- VALIDATION --------
        model.eval()
        dev_loss = 0.0
        dev_preds, dev_labels = [], []

        with torch.no_grad():
            for batch in tqdm(dev_loader, desc="Validation", leave=False):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                images = batch["pixel_values"].to(device)
                labels = batch["labels"].to(device)

                if mode == "text_only":
                    images = None

                logits = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    images=images,
                    mode=mode
                )

                loss = criterion(logits, labels)
                dev_loss += loss.item() * labels.size(0)

                preds = torch.argmax(logits, dim=1)
                dev_preds.extend(preds.cpu().numpy())
                dev_labels.extend(labels.cpu().numpy())

        dev_loss /= len(dev_loader.dataset)
        dev_f1 = f1_score(
            dev_labels,
            dev_preds,
            average="binary",
            pos_label=1,
            zero_division=0
        )

        current_lr = scheduler.get_last_lr()[0]

        history["train_loss"].append(train_loss)
        history["train_f1"].append(train_f1)
        history["dev_loss"].append(dev_loss)
        history["dev_f1"].append(dev_f1)
        history["learning_rates"].append(current_lr)

        print(f"TRAIN LOSS: {train_loss:.4f} | F1: {train_f1:.4f}")
        print(f"DEV   LOSS: {dev_loss:.4f} | F1: {dev_f1:.4f}")
        print(f"LR: {current_lr:.2e}")

        # -------- EARLY STOPPING --------
        if dev_f1 > best_f1:
            best_f1 = dev_f1
            best_model_state = {
                k: v.cpu().clone()
                for k, v in model.state_dict().items()
            }
            epochs_without_improvement = 0
            print(f"  New best Dev F1: {best_f1:.4f}")
        else:
            epochs_without_improvement += 1
            print(f"No improvement for {epochs_without_improvement} epoch(s)")

            if epochs_without_improvement >= patience:
                print("\nEarly stopping triggered!")
                break

    # LOAD BEST MODEL
    model.load_state_dict(best_model_state)
    model = model.to(device)

    print(f"\n{'=' * 60}")
    print("Training complete!")
    print(f"Best Dev F1: {best_f1:.4f}")
    print(f"{'=' * 60}")

    return model, history

### Evaluation 

In [12]:
def evaluate_multimodal_model(model, test_loader, device=DEVICE, mode="multimodal"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            if mode == "text_only":
                images = None

            logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                images=images,
                mode=mode
            )

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    metrics = {
        "accuracy": accuracy_score(all_labels, all_preds),
        "f1": f1_score(all_labels, all_preds, average="binary", pos_label=1, zero_division=0),
        "precision": precision_score(all_labels, all_preds, pos_label=1, zero_division=0),
        "recall": recall_score(all_labels, all_preds, pos_label=1, zero_division=0),
        "report": classification_report(all_labels, all_preds, zero_division=0)
    }

    return metrics

### We initialize our best model

In [13]:
BEST_FUSION = "mean"

print(f"Training BEST MULTIMODAL MODEL | Fusion = {BEST_FUSION}")

model_best = MultimodalBaseline(
    text_model_name=TEXT_MODEL_NAME,
    vision_model_name=VISION_MODEL_NAME,
    num_classes=2,
    freeze_text=False,
    freeze_vision=True,
    fusion_type=BEST_FUSION).to(DEVICE)

Training BEST MULTIMODAL MODEL | Fusion = mean
Loading TEXT encoder: microsoft/deberta-v3-base
Loading VISION encoder: openai/clip-vit-base-patch32
CLIP encoder partially frozen (last layer trainable)
MultimodalBaseline initialized | Fusion = mean
  Fused embedding size: 768


### We Train it

In [14]:
model_best, history_best = train_multimodal_model(
    model=model_best,
    train_loader=train_loader,
    dev_loader=dev_loader,
    num_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    warmup_ratio=WARMUP_RATIO,
    mode="multimodal",
    patience=PATIENCE,
    device=DEVICE)

Class weights: oppose=2.000, support=2.000
Total training steps: 2055
Warmup steps: 205

Epoch 1/15


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Training:   0%|          | 0/137 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.2048509120941162
vision_emb mean abs: 0.08117006719112396
vision_emb std: 0.11151621490716934
text_emb mean abs: 0.20609575510025024
vision_emb mean abs: 0.08293940126895905
vision_emb std: 0.11199025064706802
text_emb mean abs: 0.20471327006816864
vision_emb mean abs: 0.08206917345523834
vision_emb std: 0.11062314361333847
text_emb mean abs: 0.20317797362804413
vision_emb mean abs: 0.08343014121055603
vision_emb std: 0.11516547948122025
text_emb mean abs: 0.2089349925518036
vision_emb mean abs: 0.08685779571533203
vision_emb std: 0.11998621374368668
text_emb mean abs: 0.20612812042236328
vision_emb mean abs: 0.08353877812623978
vision_emb std: 0.11484096944332123
text_emb mean abs: 0.2059899866580963
vision_emb mean abs: 0.08494590222835541
vision_emb std: 0.11719385534524918
text_emb mean abs: 0.20516125857830048
vision_emb mean abs: 0.07961411774158478
vision_emb std: 0.10788364708423615
text_emb mean abs: 0.20615020394325256
vision_emb mean abs: 0.0795161649584

...

Validation:   0%|          | 0/13 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6813371181488037
vision_emb mean abs: 0.09414975345134735
vision_emb std: 0.12818792462348938
text_emb mean abs: 0.6615511178970337
vision_emb mean abs: 0.09186656028032303
vision_emb std: 0.12557978928089142
text_emb mean abs: 0.6705085635185242
vision_emb mean abs: 0.0906149297952652
vision_emb std: 0.12368934601545334
text_emb mean abs: 0.678033709526062
vision_emb mean abs: 0.09137307107448578
vision_emb std: 0.12584932148456573
text_emb mean abs: 0.6709140539169312
vision_emb mean abs: 0.08697645366191864
vision_emb std: 0.11726205796003342
text_emb mean abs: 0.6620450019836426
vision_emb mean abs: 0.08883468806743622
vision_emb std: 0.1210421696305275


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4fdf2bd870>
Traceback (most recent call last):
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4fdf2bd870>
Traceback (most recent call last):
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/home/dzuniga/.conda/envs/multimo

text_emb mean abs: 0.7102551460266113
vision_emb mean abs: 0.08791433274745941
vision_emb std: 0.1193007081747055
text_emb mean abs: 0.696753978729248
vision_emb mean abs: 0.09316950291395187
vision_emb std: 0.1288047432899475
text_emb mean abs: 0.6971840262413025
vision_emb mean abs: 0.08946600556373596
vision_emb std: 0.12188155949115753
text_emb mean abs: 0.7120539546012878
vision_emb mean abs: 0.08696603775024414
vision_emb std: 0.119949109852314
text_emb mean abs: 0.675845742225647
vision_emb mean abs: 0.08865547180175781
vision_emb std: 0.12135425209999084
text_emb mean abs: 0.6960842609405518
vision_emb mean abs: 0.08871132880449295
vision_emb std: 0.12110342085361481
text_emb mean abs: 0.6968250274658203
vision_emb mean abs: 0.08738420158624649
vision_emb std: 0.11962977796792984
TRAIN LOSS: 0.0088 | F1: 0.9977
DEV   LOSS: 0.6050 | F1: 0.8714
LR: 8.89e-06
No improvement for 5 epoch(s)

Early stopping triggered!

Training complete!
Best Dev F1: 0.8980


We analyzed the scale of the embeddings per each modality to understand their contribution during early fusion.

We observed that the text embeddings have a mean absolute value of aprox 0.20, while vision embedd have a lower mean around 0.08. Variance are comparable.

This is our first indicator that a clear scale imbalance, this translate  that the textual modality numerically dominates the overall representation. This is an initial explanation for why the image signal is present but underweighted in predictions.

### We evaulate it (DEV y TEST)

In [16]:
def evaluate_multimodal(model, dataloader, threshold=0.5, mode="multimodal", device=DEVICE, verbose=True):
    model.eval()
    all_labels, all_probs, all_preds = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", disable=not verbose):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            images = batch["pixel_values"].to(device)
            if mode == "text_only":
                images = None

            logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                images=images,
                mode=mode
            )

            probs = torch.softmax(logits, dim=1)[:, 1]
            preds = (probs >= threshold).long()

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    y_true = np.array(all_labels)
    y_prob = np.array(all_probs)
    y_pred = np.array(all_preds)

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, pos_label=1, zero_division=0),
        "recall": recall_score(y_true, y_pred, pos_label=1, zero_division=0),
        "f1": f1_score(y_true, y_pred, average="binary", pos_label=1, zero_division=0),
        "y_true": y_true,
        "y_pred": y_pred,
        "y_prob": y_prob
    }



In [17]:
print("\nEvaluating BEST MODEL on DEV...")
dev_metrics_best = evaluate_multimodal(model_best, dev_loader, threshold=0.5, mode="multimodal", device=DEVICE, verbose=False)

print("DEV RESULTS")
print(f"  F1:        {dev_metrics_best['f1']:.4f}")
print(f"  Precision: {dev_metrics_best['precision']:.4f}")
print(f"  Recall:    {dev_metrics_best['recall']:.4f}")
print(f"  Accuracy:  {dev_metrics_best['accuracy']:.4f}")

print("\nEvaluating BEST MODEL on TEST...")
test_metrics_best = evaluate_multimodal(model_best, test_loader, threshold=0.5, mode="multimodal", device=DEVICE, verbose=False)

print("TEST RESULTS")
print(f"  F1:        {test_metrics_best['f1']:.4f}")
print(f"  Precision: {test_metrics_best['precision']:.4f}")
print(f"  Recall:    {test_metrics_best['recall']:.4f}")
print(f"  Accuracy:  {test_metrics_best['accuracy']:.4f}")



Evaluating BEST MODEL on DEV...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6158043742179871
vision_emb mean abs: 0.09213485568761826
vision_emb std: 0.12551940977573395
text_emb mean abs: 0.5939433574676514
vision_emb mean abs: 0.08993290364742279
vision_emb std: 0.12309751659631729
text_emb mean abs: 0.623732328414917
vision_emb mean abs: 0.08853074163198471
vision_emb std: 0.12095756828784943
text_emb mean abs: 0.5991951823234558
vision_emb mean abs: 0.08965321630239487
vision_emb std: 0.12368980795145035
text_emb mean abs: 0.620233416557312
vision_emb mean abs: 0.08542454987764359
vision_emb std: 0.11522895842790604
text_emb mean abs: 0.6004571318626404
vision_emb mean abs: 0.08678548038005829
vision_emb std: 0.11845491081476212
text_emb mean abs: 0.6590863466262817
vision_emb mean abs: 0.08645844459533691
vision_emb std: 0.11737557500600815
text_emb mean abs: 0.6573745012283325
vision_emb mean abs: 0.0918998047709465
vision_emb std: 0.12711486220359802
text_emb mean abs: 0.6346981525421143
vision_emb mean abs: 0.08815988898277283
visi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
vision_emb std: 0.11793818324804306
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.08480556309223175
vision_emb std: 0.11642610281705856
text_emb mean abs: 0.6237055659294128
vision_emb mean abs: 0.090653195977211
vision_emb std: 0.12518759071826935
text_emb mean abs: 0.6034646034240723
vision_emb mean abs: 0.09529808163642883
vision_emb std: 0.12989959120750427
text_emb mean abs: 0.6351428031921387
vision_emb mean abs: 0.08960899710655212
visio

In [18]:
# 1) Text-only eval of your multimodal-trained model
text_only_from_multimodal = evaluate_multimodal(
    model_best, test_loader,
    threshold=0.5,
    mode="text_only",   # <- disables vision path
    device=DEVICE,
    verbose=False
)

# 2) Multimodal eval (normal)
multimodal_eval = evaluate_multimodal(
    model_best, test_loader,
    threshold=0.5,
    mode="multimodal",
    device=DEVICE,
    verbose=False
)

print("\n[COMPARE: multimodal-trained model]")
print(f"Multimodal eval F1: {multimodal_eval['f1']:.4f}")
print(f"Text-only eval F1:  {text_only_from_multimodal['f1']:.4f}")
print("ΔF1 (multi - text_only):", multimodal_eval["f1"] - text_only_from_multimodal["f1"])


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6237055659294128
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6034646034240723
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6351428031921387
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6224915385246277
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6671354174613953
vision_emb mean abs: 0.0
vision_emb std: 0.0
text_emb mean abs: 0.6543421745300293
vision_emb mean abs: 0.0
vision_emb std: 0.0
text

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
vision_emb std: 0.11793818324804306
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.08480556309223175
vision_emb std: 0.11642610281705856
text_emb mean abs: 0.6237055659294128
vision_emb mean abs: 0.090653195977211
vision_emb std: 0.12518759071826935
text_emb mean abs: 0.6034646034240723
vision_emb mean abs: 0.09529808163642883
vision_emb std: 0.12989959120750427
text_emb mean abs: 0.6351428031921387
vision_emb mean abs: 0.08960899710655212
visio

## Image Ablation 

In [17]:
def make_neutral_images(pixel_values: torch.Tensor, strategy="zeros"):
    """
    pixel_values: [B, 3, 224, 224] (ya en espacio CLIP-normalizado)
    strategy:
      - "zeros": tensor de ceros
      - "noise": ruido gaussiano leve
    """
    if strategy == "zeros":
        return torch.zeros_like(pixel_values)
    elif strategy == "noise":
        return torch.randn_like(pixel_values) * 0.05
    else:
        raise ValueError("strategy must be 'zeros' or 'noise'")


In [18]:
def evaluate_image_ablation(model, dataloader, threshold=0.5, image_mode="real", device=DEVICE, verbose=True):
    """
    image_mode:
      - "real": usa pixel_values reales
      - "zeros": reemplaza por ceros
      - "noise": reemplaza por ruido
    """
    model.eval()
    all_labels, all_probs, all_preds = [], [], []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Eval image={image_mode}", disable=not verbose):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            images = batch["pixel_values"].to(device)

            if image_mode == "zeros":
                images = make_neutral_images(images, strategy="zeros")
            elif image_mode == "noise":
                images = make_neutral_images(images, strategy="noise")
            elif image_mode == "real":
                pass
            else:
                raise ValueError("image_mode must be real/zeros/noise")

            logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                images=images,
                mode="multimodal"
            )

            probs = torch.softmax(logits, dim=1)[:, 1]
            preds = (probs >= threshold).long()

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    y_true = np.array(all_labels)
    y_pred = np.array(all_preds)

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, pos_label=1, zero_division=0),
        "recall": recall_score(y_true, y_pred, pos_label=1, zero_division=0),
        "f1": f1_score(y_true, y_pred, pos_label=1, zero_division=0),
    }


In [19]:
print("IMAGE ABLATION STUDY (TEST)")

img_real  = evaluate_image_ablation(model_best, test_loader, threshold=0.5, image_mode="real",  device=DEVICE, verbose=False)
img_zeros = evaluate_image_ablation(model_best, test_loader, threshold=0.5, image_mode="zeros", device=DEVICE, verbose=False)
img_noise = evaluate_image_ablation(model_best, test_loader, threshold=0.5, image_mode="noise", device=DEVICE, verbose=False)

print("\n[IMAGE ABLATION RESULTS - TEST]")
print(f"Image REAL  | F1: {img_real['f1']:.4f} | P: {img_real['precision']:.4f} | R: {img_real['recall']:.4f} | Acc: {img_real['accuracy']:.4f}")
print(f"Image ZEROS | F1: {img_zeros['f1']:.4f} | P: {img_zeros['precision']:.4f} | R: {img_zeros['recall']:.4f} | Acc: {img_zeros['accuracy']:.4f}")
print(f"Image NOISE | F1: {img_noise['f1']:.4f} | P: {img_noise['precision']:.4f} | R: {img_noise['recall']:.4f} | Acc: {img_noise['accuracy']:.4f}")

print("\n Diff F1 (REAL - ZEROS):", (img_real["f1"] - img_zeros["f1"]))
print(" Diff F1 (REAL - NOISE):", (img_real["f1"] - img_noise["f1"]))


IMAGE ABLATION STUDY (TEST)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9309351870>
Traceback (most recent call last):
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-pack

text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
vision_emb std: 0.11793818324804306
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.08480556309223175
vision_emb std: 0.11642610281705856
text_emb mean abs: 0.6237055659294128
vision_emb mean abs: 0.090653195977211
vision_emb std: 0.12518759071826935
text_emb mean abs: 0.6034646034240723
vision_emb mean abs: 0.09529808163642883
vision_emb std: 0.12989959120750427
text_emb mean abs: 0.6351428031921387
vision_emb mean abs: 0.08960899710655212
visio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6237055659294128
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6034646034240723
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6351428031921387
vision_emb mean abs: 0.07355128228664398
v

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.07235313951969147
vision_emb std: 0.10403428226709366
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.07263162732124329
vision_emb std: 0.1042366698384285
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.07256864756345749
vision_emb std: 0.10415267199277878
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.07247087359428406
vision_emb std: 0.10429278016090393
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.07231730967760086
vision_emb std: 0.10416343063116074
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.07227588444948196
vision_emb std: 0.10455428808927536
text_emb mean abs: 0.6237055659294128
vision_emb mean abs: 0.07243280857801437
vision_emb std: 0.10447987169027328
text_emb mean abs: 0.6034646034240723
vision_emb mean abs: 0.0719461441040039
vision_emb std: 0.10392461717128754
text_emb mean abs: 0.6351428031921387
vision_emb mean abs: 0.07212156057357788
vis

Here we evaluated the imapct of the visual modality by replacing real images with either zero-valued images or random noise at the test time. 

All configurations yield identical performance metrics, including our goal metric F1-Score.

This indicates that while the model is multimodal, the final predictions are not sensitive to the visual input. It suggests that text alone is enough to drive the overall classification for most samples.

In [20]:
model_best.eval()
batch = next(iter(test_loader))

input_ids = batch["input_ids"][:4].to(DEVICE)
attention_mask = batch["attention_mask"][:4].to(DEVICE)

img_real  = batch["pixel_values"][:4].to(DEVICE)
img_zeros = torch.zeros_like(img_real)
img_noise = torch.randn_like(img_real) * 0.05

with torch.no_grad():
    logits_real  = model_best(input_ids=input_ids, attention_mask=attention_mask, images=img_real,  mode="multimodal")
    logits_zeros = model_best(input_ids=input_ids, attention_mask=attention_mask, images=img_zeros, mode="multimodal")
    logits_noise = model_best(input_ids=input_ids, attention_mask=attention_mask, images=img_noise, mode="multimodal")

print("diff logits (real vs zeros):", (logits_real - logits_zeros).abs().mean().item())
print("diff logits (real vs noise):", (logits_real - logits_noise).abs().mean().item())

probs_real  = torch.softmax(logits_real, dim=1)[:, 1]
probs_zeros = torch.softmax(logits_zeros, dim=1)[:, 1]
probs_noise = torch.softmax(logits_noise, dim=1)[:, 1]

print("diff probs (real vs zeros):", (probs_real - probs_zeros).abs().mean().item())
print("dff probs (real vs noise):", (probs_real - probs_noise).abs().mean().item())
print("probs_real :", probs_real.detach().cpu().numpy())
print("probs_zeros:", probs_zeros.detach().cpu().numpy())
print("probs_noise:", probs_noise.detach().cpu().numpy())


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6394039392471313
vision_emb mean abs: 0.089239701628685
vision_emb std: 0.12085746228694916
text_emb mean abs: 0.6394039392471313
vision_emb mean abs: 0.07355131208896637
vision_emb std: 0.10103269666433334
text_emb mean abs: 0.6394039392471313
vision_emb mean abs: 0.07264067232608795
vision_emb std: 0.10395847260951996
diff logits (real vs zeros): 0.017505735158920288
diff logits (real vs noise): 0.016720890998840332
diff probs (real vs zeros): 2.7063288143835962e-05
dff probs (real vs noise): 2.9586502932943404e-05
probs_real : [8.6996157e-04 1.0333699e-03 9.9930203e-01 8.6426234e-04]
probs_zeros: [9.1944553e-04 1.0540352e-03 9.9929452e-01 8.9485612e-04]
probs_noise: [9.2404208e-04 1.0611285e-03 9.9930024e-01 8.9898118e-04]


To verify that the visual modality is not entirely ignored, we compared liguts and predicted probabilities between real and corrupted images. 

We observed small but non-zero differences in logits and probabilities they were even smaller. This confirms that the image does influence the internal model, however these changes are too small to affect the final predicted class.

In [21]:
import torch.nn.functional as F

def evaluate_image_ablation_detailed(model, dataloader, image_mode="real", device=DEVICE):
    model.eval()
    n = 0
    flip_count = 0
    delta_prob_sum = 0.0
    delta_logit_sum = 0.0
    delta_loss_sum = 0.0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Image ablation detailed: {image_mode}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            img_real = batch["pixel_values"].to(device)

            if image_mode == "zeros":
                img_alt = torch.zeros_like(img_real)
            elif image_mode == "noise":
                img_alt = torch.randn_like(img_real) * 0.05
            elif image_mode == "real":
                img_alt = img_real
            else:
                raise ValueError("image_mode must be real/zeros/noise")

            # logits con real
            logits_real = model(input_ids=input_ids, attention_mask=attention_mask, images=img_real, mode="multimodal")
            probs_real = torch.softmax(logits_real, dim=1)[:, 1]
            preds_real = torch.argmax(logits_real, dim=1)

            # logits con alt
            logits_alt = model(input_ids=input_ids, attention_mask=attention_mask, images=img_alt, mode="multimodal")
            probs_alt = torch.softmax(logits_alt, dim=1)[:, 1]
            preds_alt = torch.argmax(logits_alt, dim=1)

            # flips
            flip_count += (preds_real != preds_alt).sum().item()

            # deltas promedio
            delta_logit_sum += (logits_real - logits_alt).abs().mean().item() * labels.size(0)
            delta_prob_sum += (probs_real - probs_alt).abs().mean().item() * labels.size(0)

            # delta loss (CE)
            loss_real = F.cross_entropy(logits_real, labels)
            loss_alt = F.cross_entropy(logits_alt, labels)
            delta_loss_sum += (loss_alt - loss_real).item() * labels.size(0)

            n += labels.size(0)

    return {
        "flip_rate": flip_count / n,
        "delta_logits_meanabs": delta_logit_sum / n,
        "delta_probs_meanabs": delta_prob_sum / n,
        "delta_loss_mean": delta_loss_sum / n,  # positivo = empeora al perturbar la imagen
        "n": n
    }


In [22]:
img_zeros = evaluate_image_ablation_detailed(model_best, test_loader, image_mode="zeros")
img_noise = evaluate_image_ablation_detailed(model_best, test_loader, image_mode="noise")

print("\n[IMAGE ABLATION - DETAILED]")
print("ZEROS:", img_zeros)
print("NOISE:", img_noise)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Image ablation detailed: zeros:   0%|          | 0/19 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
vis

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Image ablation detailed: noise:   0%|          | 0/19 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.0724213570356369
vision_emb std: 0.10341425985097885
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.07282601296901703
vision_emb std: 0.1043110266327858
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.07230425626039505
vision_emb std: 0.10394826531410217
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.07248221337795258
vision_emb std: 0.10401815176010132
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
visio

To test if the images had an impact near the decision boundary, we focus on certain samples that their predictions were close to 0.5. The flip rate was 0, indicating that visual information is insufficient to change the textual signal even for ambigious cases.

This only shows that the text model alone is the one deciding the overall performance.

In [23]:
def image_flip_rate_on_uncertain(model, dataloader, low=0.4, high=0.6, device=DEVICE):
    model.eval()
    flip = 0
    total = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Flip rate on uncertain"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            img_real = batch["pixel_values"].to(device)

            logits_real = model(input_ids=input_ids, attention_mask=attention_mask, images=img_real, mode="multimodal")
            probs_real = torch.softmax(logits_real, dim=1)[:, 1]

            mask = (probs_real >= low) & (probs_real <= high)
            if mask.sum() == 0:
                continue

            # real preds
            preds_real = (probs_real >= 0.5).long()

            # zeros preds
            img_zeros = torch.zeros_like(img_real)
            logits_zeros = model(input_ids=input_ids, attention_mask=attention_mask, images=img_zeros, mode="multimodal")
            probs_zeros = torch.softmax(logits_zeros, dim=1)[:, 1]
            preds_zeros = (probs_zeros >= 0.5).long()

            flip += (preds_real[mask] != preds_zeros[mask]).sum().item()
            total += mask.sum().item()

    return {"flip_rate_uncertain": flip / max(total, 1), "n_uncertain": total}


In [24]:
uncertain_stats = image_flip_rate_on_uncertain(
    model=model_best,
    dataloader=test_loader,
    low=0.4,
    high=0.6,
    device=DEVICE
)

print("\n[IMAGE FLIP RATE ON UNCERTAIN SAMPLES]")
print(f"Flip rate: {uncertain_stats['flip_rate_uncertain']:.4f}")
print(f"N uncertain samples: {uncertain_stats['n_uncertain']}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Flip rate on uncertain:   0%|          | 0/19 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
vision_emb std: 0.11793818324804306
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.07355128228664398
vision_emb std: 0.10101412981748581
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.08480556309223175
vis

Taking into consideration all these analyses shoed that the multiodal does use visual information, but only as a very weak auxiliary signal. The textual modality dominates the representation space.

In the next section we show more clearly the performance without text model and how this is related to this conclusion.

## Text Ablation

In [25]:
def make_text_zeros(input_ids, attention_mask, pad_token_id):
    """
    Mantiene CLS/SEP si existen y el resto PAD; attention_mask solo activa esos tokens.
    input_ids: [B, L]
    """
    x = input_ids.clone()
    m = attention_mask.clone()

    # todo PAD
    x[:] = pad_token_id
    m[:] = 0

    # activamos primer token como "CLS placeholder"
    x[:, 0] = input_ids[:, 0]
    m[:, 0] = 1

    # si hay SEP típico de HF al final de secuencia real (no siempre), intentamos preservarlo:
    # buscamos el último token con mask=1 en el original
    last_real = attention_mask.sum(dim=1) - 1  # idx último token real
    for i in range(x.size(0)):
        idx = last_real[i].item()
        if idx > 0 and idx < x.size(1):
            x[i, 1] = input_ids[i, idx]  # ponemos SEP en pos 1 (compacto)
            m[i, 1] = 1

    return x, m


In [26]:
def permute_tokens(input_ids, attention_mask):
    """
    Permuta tokens SOLO donde attention_mask=1. Preserva longitud.
    """
    x = input_ids.clone()
    B, L = x.size()

    for i in range(B):
        idxs = torch.where(attention_mask[i] == 1)[0]
        if idxs.numel() <= 2:
            continue
        # preserva primer token (CLS) y último token real (SEP) por estabilidad
        inner = idxs[1:-1]
        perm = inner[torch.randperm(inner.numel(), device=inner.device)]
        x[i, inner] = x[i, perm]

    return x


In [27]:
def dropout_tokens(input_ids, attention_mask, pad_token_id, drop_prob=0.3):
    """
    Apaga tokens con prob drop_prob donde mask=1, excepto CLS y último token real.
    """
    x = input_ids.clone()
    m = attention_mask.clone()
    B, L = x.size()

    for i in range(B):
        idxs = torch.where(m[i] == 1)[0]
        if idxs.numel() <= 2:
            continue
        # no tocar CLS ni último real
        protected = torch.tensor([idxs[0].item(), idxs[-1].item()], device=idxs.device)
        candidates = idxs[1:-1]

        if candidates.numel() == 0:
            continue

        drop_mask = (torch.rand(candidates.numel(), device=candidates.device) < drop_prob)
        to_drop = candidates[drop_mask]

        x[i, to_drop] = pad_token_id
        m[i, to_drop] = 0

    return x, m


In [28]:
import torch.nn.functional as F

def evaluate_text_ablation(model, dataloader, tokenizer, ablation="zeros", drop_prob=0.3, threshold=0.5, device=DEVICE):
    model.eval()
    pad_id = tokenizer.pad_token_id

    # acumuladores
    all_labels = []
    all_preds_base = []
    all_preds_abl = []

    loss_base_sum = 0.0
    loss_abl_sum = 0.0
    n = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Text ablation: {ablation}", leave=False):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            images = batch["pixel_values"].to(device)
            labels = batch["labels"].to(device)

            # ===== BASELINE (texto real) =====
            logits_base = model(input_ids=input_ids, attention_mask=attention_mask, images=images, mode="multimodal")
            probs_base = torch.softmax(logits_base, dim=1)[:, 1]
            preds_base = (probs_base >= threshold).long()

            # ===== ABLATED TEXT =====
            if ablation == "zeros":
                x_abl, m_abl = make_text_zeros(input_ids, attention_mask, pad_id)

            elif ablation == "permute":
                x_abl = permute_tokens(input_ids, attention_mask)
                m_abl = attention_mask

            elif ablation == "dropout":
                x_abl, m_abl = dropout_tokens(input_ids, attention_mask, pad_id, drop_prob=drop_prob)

            else:
                raise ValueError("ablation must be: zeros | permute | dropout")

            logits_abl = model(input_ids=x_abl, attention_mask=m_abl, images=images, mode="multimodal")
            probs_abl = torch.softmax(logits_abl, dim=1)[:, 1]
            preds_abl = (probs_abl >= threshold).long()

            # ===== métricas =====
            all_labels.extend(labels.cpu().numpy())
            all_preds_base.extend(preds_base.cpu().numpy())
            all_preds_abl.extend(preds_abl.cpu().numpy())

            # ===== loss deltas =====
            loss_base = F.cross_entropy(logits_base, labels)
            loss_abl = F.cross_entropy(logits_abl, labels)

            loss_base_sum += loss_base.item() * labels.size(0)
            loss_abl_sum += loss_abl.item() * labels.size(0)
            n += labels.size(0)

    y_true = np.array(all_labels)
    y_pred_abl = np.array(all_preds_abl)
    y_pred_base = np.array(all_preds_base)

    flip_rate = (y_pred_abl != y_pred_base).mean()

    results = {
        "accuracy": accuracy_score(y_true, y_pred_abl),
        "precision": precision_score(y_true, y_pred_abl, pos_label=1, zero_division=0),
        "recall": recall_score(y_true, y_pred_abl, pos_label=1, zero_division=0),
        "f1": f1_score(y_true, y_pred_abl, pos_label=1, zero_division=0),
        "flip_rate_vs_base": float(flip_rate),
        "delta_loss_mean": float((loss_abl_sum - loss_base_sum) / max(n, 1)),
        "loss_base_mean": float(loss_base_sum / max(n, 1)),
        "loss_abl_mean": float(loss_abl_sum / max(n, 1)),
        "n": n
    }
    return results


In [29]:
print("\nSTEP 3) TEXT ABLATION STUDY (TEST)")

ablations = [
    ("zeros",   {}),
    ("permute", {}),
    ("dropout", {"drop_prob": 0.3})
]

text_ablation_results = {}

for abl, kwargs in ablations:
    res = evaluate_text_ablation(
        model=model_best,
        dataloader=test_loader,
        tokenizer=tokenizer,
        ablation=abl,
        threshold=0.5,
        device=DEVICE,
        **kwargs
    )
    text_ablation_results[abl] = res
    print(f"\n[TEXT ABLATION: {abl}]")
    print(f"F1: {res['f1']:.4f} | P: {res['precision']:.4f} | R: {res['recall']:.4f} | Acc: {res['accuracy']:.4f}")
    print(f"flip_rate_vs_base: {res['flip_rate_vs_base']:.4f}")
    print(f"Δloss_mean: {res['delta_loss_mean']:.6f} (base={res['loss_base_mean']:.6f} -> abl={res['loss_abl_mean']:.6f})")



STEP 3) TEXT ABLATION STUDY (TEST)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Text ablation: zeros:   0%|          | 0/19 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.3437989354133606
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.3437989354133606
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.3437989354133606
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.3437989354133606
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
visio

...

text_emb mean abs: 0.6264222860336304
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664
text_emb mean abs: 0.6567755341529846
vision_emb mean abs: 0.09199535846710205
vision_emb std: 0.12415170669555664


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9309351870>


text_emb mean abs: 0.6100606918334961
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703
text_emb mean abs: 0.6379097104072571
vision_emb mean abs: 0.08696801960468292
vision_emb std: 0.11822356283664703


Traceback (most recent call last):
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9309351870>
Traceback (most recent call last):
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/home/dzuniga/.conda/envs/multimodal/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_

text_emb mean abs: 0.6201836466789246
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6369141936302185
vision_emb mean abs: 0.08644196391105652
vision_emb std: 0.1193089559674263
text_emb mean abs: 0.6378681659698486
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6512083411216736
vision_emb mean abs: 0.08931045234203339
vision_emb std: 0.1224636510014534
text_emb mean abs: 0.6139541864395142
vision_emb mean abs: 0.08768294751644135
vision_emb std: 0.11793818324804306
text_emb mean abs: 0.6184197664260864
vision_emb mean abs: 0.08768294751644135
vision_emb std: 0.11793818324804306
text_emb mean abs: 0.6213090419769287
vision_emb mean abs: 0.08480556309223175
vision_emb std: 0.11642610281705856
text_emb mean abs: 0.6254492998123169
vision_emb mean abs: 0.08480556309223175
vision_emb std: 0.11642610281705856
text_emb mean abs: 0.6237055659294128
vision_emb mean abs: 0.090653195977211
vision_

- Text Ablation (Zeros)

When text is removed almost completly, the model collapses. F1 drops to 0.0, with a flip rate of 51.7 %. The model accuracy (0.61%) shows that the models only shows the mayoritary class. Images by themselve dont have enough semantic information.

- Text Ablation (Permutation)

If we change some tokens of order, preserving the vocabulary but destroyin the order the F1 decreases to 56% with a flip rate of 48.3 %. This shows that the model does not function like a simple bag-of-words, the structure sintactic and contextual of the text is critic for prediction.

- Text Ablation (Drop out)

With a textual dropout (30 % removed tokens), the model shows a moderate decrease in the performance F1 = 0.65  and flip rate = 28.3. This suggest that the model can tolarete some level of misinformation.94)94)